# Programming C/C++ On Linux



## 1 C/C++

The working directory: `/home/notebook/demo`

In [ ]:
%pwd

In [ ]:
%cd demo

### 1.1 Bisection Methond in C

𝑦=𝑓(𝑥) 

* ./include/roots.h

* ./src/roots.c

In [ ]:
%%file ./include/roots.h
#ifndef ROOTS_H
#define ROOTS_H

#include <math.h>
// function definition
typedef double (*fun)(double);

int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb);
/*
	The program uses the bisection method to solve the equation
		f(x)-y = 0.
	The solution is to be in [x1,x2] and it is assumed that
		(f(x1)-y)*(f(x2)-y) <= 0.
	The solution is returned in rtb, and it is to be in error by at most xacc.
	
	return value is an error indicator.
	  0, the solution has been computed satisfactorily.
	  1, (f(x1)-y)*(f(x2)-y) was greater than 0, contrary to assumption 
	  2, exceeded the maximum number of iteration 
*/
#endif


In [ ]:
%%file ./src/roots.c
/*
  Numerical Recipes http://numerical.recipes
*/ 
#include <math.h>
#include "roots.h"

int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)
{
 	const int IMAX=100; // the maximum number of iteration
    float dx,f,fmid,xmid;

	f=(*func)(x1)-y;
	fmid=(*func)(x2)-y;
	if (f*fmid >= 0.0) // endpoints do not straddle y=0
       return 1; 
    // init the root value: rtb
	*rtb = f < 0.0 ? (dx=x2-x1,x1) : (dx=x1-x2,x2);
	for (int i=1;i<=IMAX;i++) {
		fmid=(*func)(xmid=(*rtb)+(dx *= 0.5))-y;
		if (fmid <= 0.0) *rtb=xmid;
		if (fabs(dx) < xacc || fmid == 0.0) 
           return 0;
  	}
    // Exceeded the maximum number of iteration
    return 2;
}


### 1.2 The caller of Bisection Methond 

$𝑓(𝑥)=𝑥^2$ 

$4=𝑓(𝑥)$

```c
int rtbis(fun func,double y,double x1, double x2, double xacc,double *rtb)

double xl, xu, epsilon, root;
ier=rtbis(fcn,y,xl, xu, epsilon, &root);
```
`&root > double *rtb`

In [ ]:
%%file ./src/demo_rtbis.c

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include "roots.h"

double fcn(double x)
{
	double result;
	result = x * x;
	return result;
}

int main()
{
	double xl, xu, epsilon, root;
	int ier;
	xl = 0.1;
	xu = 3.2;
	epsilon = 0.001;
    double y=4;
	// Calculate root
	ier=rtbis(fcn,y,xl, xu, epsilon, &root);
	// Print answers
	printf("root = %14.7e  ier = %1d", root, ier);
	return 0;
}


In [ ]:
!gcc -o ./bin/demo_rtbis ./src/demo_rtbis.c  ./src/roots.c -I./include/

In [ ]:
!./bin/demo_rtbis

**Further Reading:**

**Path separators**

* Linux: /

* Windows: \

Escape character \

* The backslash`\` is an escape character,you use to inform that the next character is special.

### 1.3 The shared library

Linux supports two classes of libraries, namely:

* Static libraries – are bound to a program statically at compile time. 
* Shared libraries – are loaded when a program is launched and loaded into memory and binding occurs at run time. 

GCC assumes that all libraries on Linux

* start with `lib`

* Static libraries : end with `.a`

* Shared libraries: end with `.so`


#### 1.3.1 creating the shared  library

using `__cdecl` calling convention

In [ ]:
!gcc -c -O3 -Wall -fPIC -o ./obj/roots.o  ./src/roots.c -I./include
!gcc -shared -o ./bin/libroots.so  ./obj/roots.o

In [ ]:
!ls -l ./bin/lib*.so

#### 1.3.2 Using the shared library

We use the same code `./src/statdemo.c` under Windows

**Linking the library**

**Windows**
```bash
    -L./bin/ -lstat
```
**Linux**
```bash
   -L./bin/ -lstat -Wl,-rpath=./bin/  
```
* **`-Wl,option`**

    Pass option as an option to the **linker**. 
    
    If option contains `commas`, it is split into multiple options at the commas. 
    
    You can use this syntax to pass an argument to the option.
      
   
* **`-rpath=dir`** 

   **Add a directory to the `runtime` library search path**. This is used when linking an executable with shared objects. 


In [ ]:
!gcc -o  ./bin/demo_rtbis ./src/demo_rtbis.c -L./bin/ -lroots -I./include -Wl,-rpath=./bin/ 

In [ ]:
!./bin/demo_rtbis

### 1.4  The standard search  Path of library and header files

#### 1.4.1 The standard search  Path of library 
There are mainly standard locations in the filesystem where a library can be placed.

* `/lib`, `/lib64`:   system startup libraries
* `/usr/lib`,`/usr/lib64`
* `/usr/local/lib`, `/usr/local/lib64`

These directory are the standard search paths of GCC linker, if your library is installed in its default location, GCC searches it automatically, so the `-L` option can be omitted 

#### 1.4.2 The standard search  Path of header 
There are mainly standard locations in the filesystem where a header file can be placed.

* `/usr/include`

These directory are the standard search paths of GCC compiler, if your header files are installed in its default location, GCC searches it automatically so the `-I` option can actually be omitted

#### 1.4.3 Using  the standard search paths

#####  Put your files into  the standard search paths

Copy your header files and library to the the standard search paths of GCC

```
$sudo cp ./include/roots.h /usr/include/roots.h
$sudo cp ./bin/libroots.so /usr/lib/libroots.so

```

![](./img/linux/copy2searchpath.jpg)

>`cp` is a command used for copying files and directories in Linux
>
>`cp [options] source dest`
>
> -f : force to do,if you don't want to be bothered by prompts or don't care about errors, use the force
>
>  * `cp -f` : overlay the existing target file without prompting

In [ ]:
 !ls -l  /usr/lib/libroots.so

In [ ]:
!ls -l /usr/include/roots.h

##### Compiling ad linker without `-I` and `-L`

In [ ]:
!gcc -o  ./bin/demo_rtbis ./src/demo_rtbis.c -L./bin/ -lroots 
!./bin/demo_rtbis

**Delete these files from these default search path**
```bash
$sudo rm -f /usr/include/roots.h
$sudo rm -f /usr/lib/libroots.so
```
>`rm` is a command-line utility for removing files and directories
>
>rm [options] name...
>
> `rm -f`: will forcibly delete files without asking

In [ ]:
!ls -l  /usr/lib/libroots.so

In [ ]:
!ls -l /usr/include/roots.h

In [ ]:
!gcc -o  ./bin/demo_rtbis ./src/demo_rtbis.c -L./bin/ -lroots

In [ ]:
!gcc -o  ./bin/demo_rtbis ./src/demo_rtbis.c -L./bin/ -lroots -I./include -Wl,-rpath=./bin/ 

In [ ]:
!./bin/rtbisApp

#### 1.4.5 makefile
```bash
sudo make -f maketosyspath.mk
```

In [ ]:
%%file ./maketosyspath.mk
BIN=./bin/
SRC=./src/
OBJ=./obj/
INC=./include/

all: makeso syspath

syspath:   
	sudo cp $(INC)roots.h /usr/include/roots.h
	sudo cp $(BIN)libroots.so /usr/lib/libroots.so      
makeso:
	gcc -c -O3 -Wall -fPIC -o $(OBJ)roots.o  $(SRC)roots.c -I$(INC)
	gcc -shared -o $(BIN)libroots.so  $(OBJ)roots.o

clean:
	sudo rm -f /usr/include/roots.h
	sudo rm -f /usr/lib/libroots.so    

In [ ]:
!gcc -o  ./bin/demo_rtbis ./src/demo_rtbis.c -L./bin/ -lroots 
!./bin/demo_rtbis

```bash
sudo make -f maketosyspath.mk clean
```

In [ ]:
!gcc -o  ./bin/demo_rtbis ./src/demo_rtbis.c -L./bin/ -lroots -I./include -Wl,-rpath=./bin/ 

## 2 Using The CoolProp Shared Library

**Change working dir to `./notebook/coolpropdemo`**

In [ ]:
%pwd

In [ ]:
%cd ./coolpropdemo

### 2.1 Managing Linux Shared Libraries

In Linux you can have **different versions** of the same shared library in the same system.

* This allows you to add a newer version of the shared library which is `not compatible` with the older library interface **without breaking** the functionality of binaries which use the older library. 

* The programs linking with the shared library **do not need to take care** about the latest version of the shared library installed in the system. Once the latest version of the shared library is installed successfully, all the programs automatically start linking to the latest version.1111

#### 2.1.1 Shared libraries Name on Linux


**soname**

Every shared library has a special name called the `soname(Short for shared object name）`. The soname has the prefix `lib`, the name of the library, the phrase `.so` followed by a version number 

The soname is used to indicate what binary **api compatibility** your library support by providing version **backwards-compatibility** information. For instance, if version 6.0.0 through 6.4.1 of the shared library `libCoolProp` provide identical interfaces, they all have the same soname, `libCoolProp.so.6` 

On a working system. a fully-qualified soname is simply `a symbolic link` to the shared library's `real name`.

The symlink is what programs using the library will look for, the running programs do not care about the **version number.**

>using `soname` to manage the version : `-Wl,-soname,`$(SONAME)`
>
>gcc -shared -fPIC -o `$(REAL_NAME)` foo.c -Wl,-soname,`$(SONAME)`
>

**real name**

Every shared library also has a `real name`, which is the filename containing the actual library code. The real name adds to the soname a  minor number and the release number. 

**linker name**

linker name is the name that the compiler uses when requesting a library, which is simply the soname **without any version number**

The linker name is simply created as `a symbolic link to the soname` 

The symlink is what programs using the library will look for, the running programs do not care about the **version number**

> **A symbolic link** is a `file` that links to another file or directory using its path. you can think of a symbolic link as `a shortcut` to a file or directory (folder). symbolic links may be used in the command line, or in a script or another program.

These naming conventions help **multiple** versions of same shared library to **co-exist** in a system.



The name used by linker is usually a symbolic link to the fully qualified soname which in turn is a symbolic link to the real name.

* `linker name(a symbolic link) > soname(a symbolic link) > real name`.

#### 2.1.2 The CoolProp Shared Library use soname to manage the version

* `libCoolProp.so.6.4.1` is the library using soname to manage the version

  * `libCoolProp.so.6.4.1` is **real name**

* `libCoolProp.so.6` is **soname**,version 6

* `libCoolProp` is **linker name**


To find the soname of the library run:

In [ ]:
!readelf -a ./bin/libCoolProp.so.6.4.1 | grep SONAME

---
**readelf  - Displays information about ELF files.**


* `-a`: quivalent to specifying --file-header, --program-headers, --sections, --symbols, --relocs, --dynamic, --notes and --version-info.

**ELF**


ELF is the abbreviation for ·Executable and Linkable Format· and defines the structure for binaries, libraries, and core files. 

The formal specification allows the operating system to interpreter its underlying machine instructions correctly. 

ELF files are typically the output of a compiler or linker and are a binary format.

**Grep**

Grep is an acronym that stands for `Global Regular Expression Print`. 

Grep is a Linux/Unix command-line tool used to search for `a string of characters` in a specified file. 

The `text search pattern` is called a regular expression.

When it finds a match, it prints the line with the result

---

**The pipe character ‘|’**

A pipe is a form of `redirection` (transfer of standard output to some other destination) that is used in Linux and other Unix-like operating systems to send the output of one command/program/process to another command/program/process for further processing. 

The Unix/Linux systems allow stdout of a command to be connected to stdin of another command. You can make it do so by using `the pipe character ‘|’`.

Syntax :

`command_1 | command_2 | command_3 | .... | command_N `

---

#### 2.1.3 Install the CoolProp to the default path on Linux

copy  `./bin/libCoolProp.so.6.4.1` to  the default path `/usr/lib`

```bash
$sudo cp ./bin/libCoolProp.so.6.4.1 /usr/lib/
```

type the following command in the terminal of `./coolpropdemo/` to create the symbolic link `libCoolProp.so.6` and `libCoolProp.so`
```bash
$sudo ln -sf libCoolProp.so.6.4.1  /usr/lib/libCoolProp.so.6
$sudo ln -sf libCoolProp.so.6  /usr/lib/libCoolProp.so
```

![installcoolprop2linux](./img/linux/installcoolprop2linux.jpg)

The soname `libCoolProp.so.6`  is a symlink which points to the real name(actual library) `libCoolProp.so.6.4.1`. 

The linker name `libCoolProp.so`  is a symlink which points to the soname `libCoolProp.so.6` 

---

**`ln`** is a command-line utility for creating **links** between files.

To create a symbolic link to a given file, use the `-s (--symbolic)` option.

```bash
 $ln -s source_file symbolic_link
```

`ln` with `-f` force option:  remove existing destination files

```bash
$ln -sf source_file symbolic_link
```
---

### 2.2 Using The CoolProp  shared library

The CoolProp shared library in the default path `/usr/lib` 

In [ ]:
%%file ./mainso.c
/* 
The example use the shared library of CoolProp in c

g++ -DCOOLPROP_LIB  -o ./bin/mainso  mainso.c -I./include -L./bin -lCoolProp 
       
./bin/mainso

*/

#include "CoolPropLib.h"
#include <stdio.h>

int main() {
     double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
     printf("%f\n", value); 
     return 0;
}

In [ ]:
!g++ -DCOOLPROP_LIB -o ./bin/mainso  mainso.c -I./include -L./bin -lCoolProp 

In [ ]:
!./bin/mainso

**MakeFile**

In [ ]:
%%file ./makefile
BIN=./bin/
INC=./include/
MAIN_FILE=mainso.c
SONAME=libCoolProp.so.6
REAL_NAME=libCoolProp.so.6.4.1
LINKER_NAME=libCoolProp.so
OUTPUT_FILE=mainso.out
LINKERFLAGS=-L./bin -lCoolProp 

all:buildexec runner

buildexec:
	g++ -DCOOLPROP_LIB -o $(BIN)${OUTPUT_FILE}  $(MAIN_FILE) -I$(INC) $(LINKERFLAGS)
    
runner:
	$(BIN)${OUTPUT_FILE}
    
install:
	cp $(BIN)$(REAL_NAME) /usr/lib/
	ln -sf $(REAL_NAME)  /usr/lib/$(SONAME)
	ln -sf $(SONAME)  /usr/lib/$(LINKER_NAME)
    

In [ ]:
!make

In [ ]:
!make buildexec

In [ ]:
!make runner

**install the coolprop  to linux**

```bash
$sudo make install
```
![installcoolprop2linux-make](./img/linux/installcoolprop2linux-make.jpg)

In [ ]:
!make -f  ./demo/makegslexample.mk

## Reference

* [Ubuntu-Python-CPP(Chinese)](https://github.com/thermalogic/home/blob/B2021/guide/doc/Ubuntu-Python-CPP(Chinese).md)

* [CoolProp Shared Library](http://www.coolprop.org/coolprop/wrappers/SharedLibrary/index.html)

